In [1]:
import pandas as pd

# === Step 1: Set the folder containing your CSV files ===
folder_path = "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/"  # <-- change this to your folder path


# === Step 2: Find all CSV files in that folder ===
csv_file_paths = [
    "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/aggregation_queries_with_df_code.csv",
    "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/sorting_ranking_queries_with_df_code.csv",
    "/Users/p0s0cad/work/personal-projects/ai-munshi/model/dataset/dataset_templates/text_string_queries_with_df_code.csv"
]

# === Step 3: Read and concatenate them ===
all_dfs = []

for file_path in csv_file_paths:
    df = pd.read_csv(file_path)
    all_dfs.append(df)

final_df = pd.concat(all_dfs, ignore_index=True)

final_df=final_df[["query_type","query","col_1","col_2","df_command"]]

# === Step 4: Save the combined dataset ===
output_path = "./dataset/combined_dataset.csv"
final_df.to_csv(output_path, index=False)

print(f"✅ Combined {len(csv_file_paths)} files into {output_path}")


✅ Combined 3 files into ./dataset/combined_dataset.csv


In [6]:
import random
import pandas as pd
import json

# === Step 0: Fake columns pool for schema noise ===
fake_cols_pool = [
    'Request ID', 'Tracking Code', 'Approval Status', 'Internal Notes', 'Timestamp Created',
    'Last Modified By', 'Error Code', 'Sync Status', 'Origin System', 'Archived Flag',
    'Workflow Step', 'Reviewer Comments', 'Flagged Reason', 'Processing Time', 'Manual Override',
    'Document Ref', 'System ID', 'Batch Number', 'Response Time (ms)', 'Audit Trail',
    'Project Tag', 'Release Version', 'Run ID', 'Environment Name', 'Retry Count',
    'Source File Name', 'Alert Triggered', 'Backup ID', 'Session Token', 'Validation Notes'
]

# === Step 1: Load your CSV ===
input_csv_path = "./dataset/combined_dataset.csv"
df = pd.read_csv(input_csv_path)

# === Step 2: Convert to CodeT5 format ===
output_data = []

for _, row in df.iterrows():
    # Collect actual columns used
    true_cols = list({col.strip() for col in [row["col_1"], row.get("col_2", None)] if pd.notna(col) and col.strip()})
    extra_cols = random.sample(fake_cols_pool, 5)
    all_cols = sorted(set(true_cols + extra_cols))
    schema = ", ".join(all_cols)

    # Create prompt: embed schema into source
    query = row["query"].strip()
    source = f"The DataFrame contains the following columns: {schema}.\nQuery: {query}"

    # The expected code output
    target = row["df_command"].strip()

    output_data.append({
        "source": source,
        "target": target
    })

# === Step 3: Save JSONL ===
output_jsonl_path = "./codet5_dataset.jsonl"
with open(output_jsonl_path, "w") as f:
    for item in output_data:
        f.write(json.dumps(item) + "\n")

print(f"✅ Saved {len(output_data)} examples to {output_jsonl_path} for CodeT5 training.")


✅ Saved 1006 examples to ./codet5_dataset.jsonl for CodeT5 training.


In [ ]:
%pip install transformers==4.51.0
%pip install accelerate==1.6.0
%pip install datasets==3.5.0

In [40]:
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
from datasets import load_dataset
import torch
import evaluate

# === Config ===
MODEL_NAME = "Salesforce/codet5p-220m"
JSONL_PATH = "codet5_dataset.jsonl"  # <-- Change this to your actual path
OUTPUT_DIR = "./codet5p-finetuned"
MAX_INPUT_LEN = 256
MAX_TARGET_LEN = 256
EPOCHS = 50
BATCH_SIZE = 8
LEARNING_RATE = 5e-5

# === Load tokenizer & model ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME,cache_dir="/tmp/codet5p")

# === Load and split JSONL dataset ===
raw_dataset = load_dataset("json", data_files={"data": JSONL_PATH})["data"]
split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# === Tokenization ===
def tokenize_function(example):
    inputs = tokenizer(
        example["source"],
        padding="max_length",
        truncation=True,
        max_length=MAX_INPUT_LEN
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target"],
            padding="max_length",
            truncation=True,
            max_length=MAX_TARGET_LEN
        )
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# === Data Collator ===
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# === Evaluation Metric ===
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=decoded_preds, references=decoded_labels)

# === Training Arguments ===
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=20,
    predict_with_generate=True,
    report_to="none",
    load_best_model_at_end=True,              # Important: load best model after training
)

# === Trainer ===
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# === Train ===
trainer.train()

# === Save Model ===
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


/var/folders/7m/fhfnftjx66qbqccx433rlq140000gp/T/ipykernel_10270/3087779227.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/Users/p0s0cad/work/personal-projects/ai-munshi/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.874600,3.319490,0.865452,0.805865,0.860228,0.860372
2,1.548300,1.160948,0.945642,0.922554,0.945389,0.945494
3,0.185300,0.123602,0.960989,0.950005,0.961069,0.960900
4,0.036500,0.025856,0.965106,0.957460,0.964938,0.964810
5,0.016800,0.013641,0.966150,0.959038,0.966056,0.965992
6,0.011000,0.009353,0.966150,0.959038,0.966056,0.965992
7,0.007900,0.006863,0.966265,0.958959,0.966109,0.966104
8,0.006000,0.007664,0.957744,0.937007,0.956289,0.956109
9,0.007700,0.005789,0.962846,0.954927,0.962952,0.962721
10,0.004100,0.004352,0.965985,0.959525,0.966235,0.966185


/Users/p0s0cad/work/personal-projects/ai-munshi/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/p0s0cad/work/personal-projects/ai-munshi/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/p0s0cad/work/personal-projects/ai-munshi/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/p0s0cad/work/personal-projects/ai-munshi/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, the

('./codet5p-finetuned/tokenizer_config.json',
 './codet5p-finetuned/special_tokens_map.json',
 './codet5p-finetuned/vocab.json',
 './codet5p-finetuned/merges.txt',
 './codet5p-finetuned/added_tokens.json',
 './codet5p-finetuned/tokenizer.json')

In [39]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model_path = "./codet5p-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

def generate_code(query, max_new_tokens=128):
    input_ids = tokenizer(query, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,            # deterministic
        num_beams=5,                # better quality (optional)
        early_stopping=True
    )

    generated_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_code

query = '''The DataFrame contains the following columns: Approval Status, Archived Flag, Document Ref, Release Version, Request ID, channel, quantity.
Query: Get the number of quantity values grouped by channel.'''
code = generate_code(query)
print("🔧 Generated Pandas Code:\n", code)


🔧 Generated Pandas Code:
 df.groupby('channel')['quantity'].agg('count').reset_index()
